In [5]:
from pathlib import Path
import os, sys
sys.path.append(str(Path(os.getcwd()).parent.parent))

### Load a forward model and generate some statements


In [6]:
from evariste.generation.cli.factories import forward_prover_from_checkpoint
from evariste.generation.forward_prover import ProverConfig, AsyncProver
from evariste.generation.cli.fwd_configs import DECODING

checkpoint_path = ""

prover, dico, params, env_helper = forward_prover_from_checkpoint(
    ckpt_path=checkpoint_path,
    device="cuda",
    # No idea what's in there, but I assume it samples rather than use a beam search
    cfg=DECODING["sampling"]
)

[MIGRATION] - Changing decoding to decoding_params prefixes in flatdict
INFO - 03/01/21 11:33:06 - 2 days, 18:58:21 - Created dictionary with 4410 words (4356 total).
INFO - 03/01/21 11:33:06 - 2 days, 18:58:21 - Model trained for 22 epoches
INFO - 03/01/21 11:33:06 - 2 days, 18:58:21 - Fp16: False


In [7]:
from evariste.generation.fwd_eq.eq_env_helper import eq_env_goal_stream
from numpy.random.mtrand import RandomState

metadata = {}
rng = RandomState()

# Define a fixed length iterator over goals.
def gs(env_helper, rng, metadata, n_goals):
    for i in range(n_goals):
        goal = env_helper.build_generation_goal(rng=rng)
        metadata[i] = {}
        yield i, goal

The goal stream defines `tasks` for the prover. We can now launch a `proof_iterator` and inspect the result.

`proof.info` tells us why the generation stopped. Here, we see that the generation stopped when a `stop_tactic` was generated. Since this is what I wanted to introduce in this [PR](https://github.com/fairinternal/Evariste/pull/376), I'm happy.

In [9]:
goal_stream = gs(env_helper, rng, metadata, 10)
proof_iterator = prover.prove(goal_stream)
all_proofs = [proof for _, proof in proof_iterator]
for proof in all_proofs:
    print(proof.info)

GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 9), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 9), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 10), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 10), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 12), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 14), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 15), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 16), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 16), solved=False)
GenerationInfos(n_invalid_consecutive=0, stopped=('stop_tactic', 17), solved=False)


Maybe we want to look at a specific generation. As expected, the last statement corresponding to the `stop_action` is None. Everything went well.

In [10]:
one_proof = all_proofs[0]
for step in one_proof.generation.forward_steps():
    print(step.statement)

add max x0 max + 6 mul tan add + 5 0 x1 + 1 2 cosh x0 <EQ_DELIMITOR> add max x0 max + 6 atanh tanh mul tan add + 5 0 x1 + 1 2 cosh x0
add max x0 max + 6 mul tan add + 5 0 x1 + 1 2 cosh x0 <EQ_DELIMITOR> add max x0 max + 6 atanh tanh mul + 1 2 tan add + 5 0 x1 cosh x0
add max x0 max + 6 mul tan add + 5 0 x1 + 1 2 cosh x0 <EQ_DELIMITOR> add max x0 max + 6 atanh tanh mul + 1 2 tan add + 5 0 x1 mul add exp x0 exp neg x0 inv + 2
add max x0 max + 6 mul tan add + 5 0 x1 + 1 2 cosh x0 <EQ_DELIMITOR> asinh sinh add max x0 max + 6 atanh tanh mul + 1 2 tan add + 5 0 x1 mul add exp x0 exp neg x0 inv + 2
add max x0 max + 6 mul tan add + 5 0 x1 + 1 2 cosh x0 <EQ_DELIMITOR> asinh sinh add max x0 max + 6 atanh tanh mul + 1 2 tan asin sin add + 5 0 x1 mul add exp x0 exp neg x0 inv + 2
add max x0 max + 6 mul tan add + 5 0 x1 + 1 2 cosh x0 <EQ_DELIMITOR> pow2 sqrt asinh sinh add max x0 max + 6 atanh tanh mul + 1 2 tan asin sin add + 5 0 x1 mul add exp x0 exp neg x0 inv + 2
add max x0 max + 6 mul tan add 